In [1]:
from datashader.utils import lnglat_to_meters
from holoviews.element.tiles import EsriImagery
import holoviews as hv, pandas as pd, colorcet as cc
import hvplot.pandas
from holoviews.element.tiles import CartoDark

import holoviews as hv
from bokeh.io import output_notebook, show

output_notebook()  # makes Bokeh plots render in notebook

import panel as pn

pn.extension()

import asyncio
import websockets

import json
import os
import re
import pandas as pd

import matplotlib.cm as cm

from datetime import datetime, timezone

Loading BokehJS ...

In [ ]:
######## Note: takes ~ 20s to run, as it pulls positionreports for 15 seconds, then reads to json, etc ###############
async def connect_ais_stream(duration = 5):
        
    async with websockets.connect("wss://stream.aisstream.io/v0/stream") as websocket:
        ais_data = []
        subscribe_message = {"APIKey": "f2db622b805da931e6b1e6609ceb659b355512ed",  # Required !
                             "BoundingBoxes": [[[-90, -180], [90, 180]]]} # Required!

        subscribe_message_json = json.dumps(subscribe_message)
        await websocket.send(subscribe_message_json)

        end_time = asyncio.get_event_loop().time() + duration

        while asyncio.get_event_loop().time() < end_time:
            try:
                message_json = await websocket.recv()
                message = json.loads(message_json)
                if message.get("MessageType") == "PositionReport":
                    ais_data.append(message['MetaData'])
            except Exception as e:
                print(f"Error receiving message: {e}")
                break
        with open(f"ais_data_{datetime.now().strftime('%m%d_%H%M')}.json",'w') as f:
            json.dump(ais_data,f)
        return(ais_data)
    
api_data_store = {}
api_data_store[f"api_data_{datetime.now().strftime('%d%m_%H%M')}"] = await connect_ais_stream(duration=15)
api_df = pd.DataFrame()
for key in api_data_store.keys():
    api_df = pd.concat([api_df, pd.DataFrame(api_data_store[key])],ignore_index=True)

#note: code taken/adapted from AISStream.io instruction page
    

In [ ]:
hv.extension('bokeh')

xlim = (-2.2e7, 2.5e7)
ylim = (-1.2e7, 1.5e7)

api_df.loc[:, 'x'], api_df.loc[:, 'y'] = lnglat_to_meters(api_df['longitude'], api_df['latitude'])

map_tiles = CartoDark().opts(alpha=0.5, width=1200, height=800, bgcolor='black', xlim=xlim, ylim=ylim)
plot = api_df.hvplot(
    'x', 
    'y', 
    kind='scatter', 
    rasterize=True, 
    cmap=cc.fire, 
    cnorm='eq_hist',  
    colorbar=True).opts(colorbar_position='bottom', title= 'Heatmap of AIS Locations', xlabel='',ylabel='')
# map_tiles * plot

app = pn.panel(map_tiles * plot)
app.servable()

### code taken and adopted from https://towardsdatascience.com/plotting-heat-maps-in-python-using-bokeh-folium-and-hvplot-eb7c7f49dbc6/ ####